In [ ]:
# Computing avg price as average of open and close price
def avg_price(open_price,close_price):
    return (float(open_price)+float(close_price))/2

# Computing total traded monetary volume(TMV) as volume * avg price
def traded_monetary_volume(volume,open_price,close_price):
    return float(volume)*avg_price(open_price,close_price)

# Computing %price change as 100*(current close price - previous record's close price)/previous record's close price
def percent_price_change(prev_close,curr_close):
    prev_close = float(prev_close)
    curr_close = float(curr_close)
    return 100*(curr_close-prev_close)/prev_close

# Computing %volume change similarly
def percent_tmv_change(prev_tmv,curr_tmv):
    prev_tmv = float(prev_tmv)
    curr_tmv = float(curr_tmv)
    return 100*(curr_tmv-prev_tmv)/prev_tmv

In [103]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('crypto10_final.csv',header=False)
df.show(5)

+---+---+--------+---+-------------------+------------------+------------------+-------------+-----------------+----------------+----+
|_c0|_c1|     _c2|_c3|                _c4|               _c5|               _c6|          _c7|              _c8|             _c9|_c10|
+---+---+--------+---+-------------------+------------------+------------------+-------------+-----------------+----------------+----+
|  0|  1|Polkadot|DOT|2020-08-21 23:59:59|     3.09009266777|2.7309186839899997|2.89692328556|     2.8750282692| 49313136.825242| 0.0|
|  1|  2|Polkadot|DOT|2020-08-22 23:59:59|4.5106133890099995|2.8167618533599996|2.87857004326|4.484690143430001| 485801095.84575| 0.0|
|  2|  3|Polkadot|DOT|2020-08-23 23:59:59|     4.48705836489|3.7162710539199995|4.48705836489|    3.96706569209|320749483.243806| 0.0|
|  3|  4|Polkadot|DOT|2020-08-24 23:59:59|     4.78315823996|3.8126695191300004|3.98741349226|    4.60261415178|407690170.881454| 0.0|
|  4|  5|Polkadot|DOT|2020-08-25 23:59:59| 5.7607753501

In [104]:
# order of columns in original dataframe
# index, sno, name, symbol, date, high, low, open, close, volume, marketcap

# new data with date, symbol, open, close, avg_price and traded monetary volume
# ordered for each currency by date
crypto_data = df.rdd.map(lambda x: (x[4],x[3],x[7],x[8], \
                                    avg_price(x[7],x[8]), \
                                    traded_monetary_volume(x[9],x[7],x[8]), \
                                    )).sortBy(lambda x: (x[1],x[0]) )
df1 = sqlContext.createDataFrame(crypto_data,['date', 'symbol', 'open', 'close', 'avg_price', 'tmv'])
df1.show(5)

In [106]:
# Using window function to find the running difference for price and tmv from the previous record 
from pyspark.sql import functions as F
from pyspark.sql.window import Window
my_window = Window.partitionBy().orderBy('symbol')
df1 = df1.withColumn("prev_symbol", F.lag(df1.symbol).over(my_window))
df1 = df1.withColumn("prev_close", F.lag(df1.close).over(my_window))
df1 = df1.withColumn("prev_tmv", F.lag(df1.tmv).over(my_window))
df1.show(5)

In [108]:
# calculating daily price change
# condition that the previous value be for the same symbol
crypto_index = df1.rdd.map(lambda x: (x[0],x[1], \
                            float(percent_price_change(x[7],x[3]) if not(x[6]!=x[1] or x[7] is None) else percent_price_change(x[2],x[3])), \
                            float(percent_tmv_change(x[8],x[5]) if not(x[6]!=x[1] or  x[8] is None) else 0)))
crypto_index_df = crypto_index.toDF(('date', 'symbol', 'price_change_percent', 'tmv_change_percent'))
crypto_index_df.show(5)

+-------------------+------+--------------------+-------------------+
|               date|symbol|price_change_percent| tmv_change_percent|
+-------------------+------+--------------------+-------------------+
|2018-07-01 23:59:59|   ADA|  2.8525101468518357|                0.0|
|2018-07-02 23:59:59|   ADA|  11.300354998634138|  59.68572995197603|
|2018-07-03 23:59:59|   ADA|  -4.816210574102062|-31.255423960496184|
|2018-07-04 23:59:59|   ADA|  0.2024472898194967| -22.10059289132683|
|2018-07-05 23:59:59|   ADA| -1.3506891984545462|-14.863575086983603|
+-------------------+------+--------------------+-------------------+
only showing top 5 rows



In [112]:
# group by date and find the avg changes in price and volume
crypto_index_final = crypto_index.map(lambda x: (x[0],(x[2],x[3],1)) ) \
            .reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1],a[2]+b[2]) ) \
            .mapValues(lambda y: (y[0]/y[2],y[1]/y[2])) \
            .map(lambda x: (x[0],x[1][0],x[1][1]))
crypto_index_final_df = crypto_index_final.toDF(('date', 'avg_price_change_percent', 'avg_tmv_change_percent'))
crypto_index_final_df.show(5)

+-------------------+------------------------+----------------------+
|               date|avg_price_change_percent|avg_tmv_change_percent|
+-------------------+------------------------+----------------------+
|2018-07-01 23:59:59|     -0.3545608063329723|                   0.0|
|2018-07-02 23:59:59|       5.121125072620959|     9.193665598572519|
|2018-07-03 23:59:59|     -1.7263876931070241|      3.88579704445608|
|2018-07-04 23:59:59|      0.4605982027758544|   -17.906456882074515|
|2018-07-05 23:59:59|     -0.7984669906309387|     5.925269446030441|
+-------------------+------------------------+----------------------+
only showing top 5 rows



In [136]:
# nasdaq100
df = sqlContext.read.csv('nasdaq100_final.csv',header=False)
df.show(5)

# order of columns in original dataframe
# reset_index, index, date, low, open, volume, high, close, adjusted_close

# new data with date, symbol, open, adjusted_close, avg_price and traded monetary volume
# ordered for each stock (already ordered by date)
stock_data = df.rdd.map(lambda x: (x[2],x[9],x[4],x[8], \
                                    avg_price(x[4],x[8]), \
                                    traded_monetary_volume(x[5],x[4],x[8]), \
                                    )).sortBy(lambda x: (x[1]) )
df1 = sqlContext.createDataFrame(stock_data,['date', 'symbol', 'open', 'close', 'avg_price', 'tmv'])
df1.show(5)
# Using window function to find the running difference for price and tmv from the previous record 
my_window = Window.partitionBy().orderBy('symbol')
df1 = df1.withColumn("prev_symbol", F.lag(df1.symbol).over(my_window))
df1 = df1.withColumn("prev_close", F.lag(df1.close).over(my_window))
df1 = df1.withColumn("prev_tmv", F.lag(df1.tmv).over(my_window))
# calculating daily price change
# condition that the previous value be for the same symbol
stock_index = df1.rdd.map(lambda x: (x[0],x[1], \
                            float(percent_price_change(x[7],x[3]) if not(x[6]!=x[1] or x[7] is None) else percent_price_change(x[2],x[3])), \
                            float(percent_tmv_change(x[8],x[5]) if not(x[6]!=x[1] or  x[8] is None) else 0)))
stock_index_df = stock_index.toDF(('date', 'symbol', 'price_change_percent', 'tmv_change_percent'))
stock_index_df.show(5)
# group by date and find the avg changes in price and volume
stock_index_final = stock_index.map(lambda x: (x[0],(x[2],x[3],1)) ) \
            .reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1],a[2]+b[2]) ) \
            .mapValues(lambda y: (y[0]/y[2],y[1]/y[2])) \
            .map(lambda x: (x[0],x[1][0],x[1][1]))
nasdaq100_df = stock_index_final.toDF(('date', 'avg_price_change_percent', 'avg_tmv_change_percent'))
nasdaq100_df.show(5)

+----+----+----------+------------------+------------------+--------+-----------------+------------------+------------------+----+
| _c0| _c1|       _c2|               _c3|               _c4|     _c5|              _c6|               _c7|               _c8| _c9|
+----+----+----------+------------------+------------------+--------+-----------------+------------------+------------------+----+
|7148|7148|02-07-2018| 42.20000076293945|42.560001373291016|21810900|42.83000183105469|42.810001373291016| 38.80120086669922|CSCO|
|7149|7149|03-07-2018| 42.61999893188477| 42.93999862670898|13474700|43.22999954223633| 42.66999816894531|38.674312591552734|CSCO|
|7150|7150|05-07-2018|41.959999084472656|42.560001373291016|18118700|42.63999938964844|42.560001373291016| 38.87525939941406|CSCO|
|7151|7151|06-07-2018|42.310001373291016| 42.38999938964844|20258300|42.88000106811523|42.709999084472656| 39.01226425170898|CSCO|
|7152|7152|09-07-2018| 42.36000061035156|             42.75|23210000|42.77999877929

In [137]:
# sp100
df = sqlContext.read.csv('sp100_final.csv',header=False)
df.show(5)

# order of columns in original dataframe
# reset_index, index, date, low, open, volume, high, close, adjusted_close

# new data with date, symbol, open, adjusted_close, avg_price and traded monetary volume
# ordered for each stock (already ordered by date)
stock_data = df.rdd.map(lambda x: (x[2],x[9],x[4],x[8], \
                                    avg_price(x[4],x[8]), \
                                    traded_monetary_volume(x[5],x[4],x[8]), \
                                    )).sortBy(lambda x: (x[1]) )
df1 = sqlContext.createDataFrame(stock_data,['date', 'symbol', 'open', 'close', 'avg_price', 'tmv'])
df1.show(5)
# Using window function to find the running difference for price and tmv from the previous record 
my_window = Window.partitionBy().orderBy('symbol')
df1 = df1.withColumn("prev_symbol", F.lag(df1.symbol).over(my_window))
df1 = df1.withColumn("prev_close", F.lag(df1.close).over(my_window))
df1 = df1.withColumn("prev_tmv", F.lag(df1.tmv).over(my_window))
# calculating daily price change
# condition that the previous value be for the same symbol
stock_index = df1.rdd.map(lambda x: (x[0],x[1], \
                            float(percent_price_change(x[7],x[3]) if not(x[6]!=x[1] or x[7] is None) else percent_price_change(x[2],x[3])), \
                            float(percent_tmv_change(x[8],x[5]) if not(x[6]!=x[1] or  x[8] is None) else 0)))
stock_index_df = stock_index.toDF(('date', 'symbol', 'price_change_percent', 'tmv_change_percent'))
stock_index_df.show(5)
# group by date and find the avg changes in price and volume
stock_index_final = stock_index.map(lambda x: (x[0],(x[2],x[3],1)) ) \
            .reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1],a[2]+b[2]) ) \
            .mapValues(lambda y: (y[0]/y[2],y[1]/y[2])) \
            .map(lambda x: (x[0],x[1][0],x[1][1]))
sp100_df = stock_index_final.toDF(('date', 'avg_price_change_percent', 'avg_tmv_change_percent'))
sp100_df.show(5)

+----+----+----------+------------------+------------------+--------+-----------------+------------------+------------------+----+
| _c0| _c1|       _c2|               _c3|               _c4|     _c5|              _c6|               _c7|               _c8| _c9|
+----+----+----------+------------------+------------------+--------+-----------------+------------------+------------------+----+
|7148|7148|02-07-2018| 42.20000076293945|42.560001373291016|21810900|42.83000183105469|42.810001373291016| 38.54083633422852|CSCO|
|7149|7149|03-07-2018| 42.61999893188477| 42.93999862670898|13474700|43.22999954223633| 42.66999816894531|38.414798736572266|CSCO|
|7150|7150|05-07-2018|41.959999084472656|42.560001373291016|18118700|42.63999938964844|42.560001373291016|38.614402770996094|CSCO|
|7151|7151|06-07-2018|42.310001373291016| 42.38999938964844|20258300|42.88000106811523|42.709999084472656|    38.75048828125|CSCO|
|7152|7152|09-07-2018| 42.36000061035156|             42.75|23210000|42.77999877929

In [138]:
# nasdaq100Tech
df = sqlContext.read.csv('nasdaq100Tech_final.csv',header=False)
df.show(5)

# order of columns in original dataframe
# reset_index, index, date, low, open, volume, high, close, adjusted_close

# new data with date, symbol, open, adjusted_close, avg_price and traded monetary volume
# ordered for each stock (already ordered by date)
stock_data = df.rdd.map(lambda x: (x[2],x[9],x[4],x[8], \
                                    avg_price(x[4],x[8]), \
                                    traded_monetary_volume(x[5],x[4],x[8]), \
                                    )).sortBy(lambda x: (x[1]) )
df1 = sqlContext.createDataFrame(stock_data,['date', 'symbol', 'open', 'close', 'avg_price', 'tmv'])
df1.show(5)
# Using window function to find the running difference for price and tmv from the previous record 
my_window = Window.partitionBy().orderBy('symbol')
df1 = df1.withColumn("prev_symbol", F.lag(df1.symbol).over(my_window))
df1 = df1.withColumn("prev_close", F.lag(df1.close).over(my_window))
df1 = df1.withColumn("prev_tmv", F.lag(df1.tmv).over(my_window))
# calculating daily price change
# condition that the previous value be for the same symbol
stock_index = df1.rdd.map(lambda x: (x[0],x[1], \
                            float(percent_price_change(x[7],x[3]) if not(x[6]!=x[1] or x[7] is None) else percent_price_change(x[2],x[3])), \
                            float(percent_tmv_change(x[8],x[5]) if not(x[6]!=x[1] or  x[8] is None) else 0)))
stock_index_df = stock_index.toDF(('date', 'symbol', 'price_change_percent', 'tmv_change_percent'))
stock_index_df.show(5)
# group by date and find the avg changes in price and volume
stock_index_final = stock_index.map(lambda x: (x[0],(x[2],x[3],1)) ) \
            .reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1],a[2]+b[2]) ) \
            .mapValues(lambda y: (y[0]/y[2],y[1]/y[2])) \
            .map(lambda x: (x[0],x[1][0],x[1][1]))
nasdaq100Tech_df = stock_index_final.toDF(('date', 'avg_price_change_percent', 'avg_tmv_change_percent'))
nasdaq100Tech_df.show(5)

+----+----+----------+-----------------+------------------+-------+------------------+------------------+-----------------+----+
| _c0| _c1|       _c2|              _c3|               _c4|    _c5|               _c6|               _c7|              _c8| _c9|
+----+----+----------+-----------------+------------------+-------+------------------+------------------+-----------------+----+
|8524|8524|02-07-2018| 95.0500030517578| 95.45999908447266|2029100|  97.1999969482422| 97.05999755859376|92.29022979736328|SWKS|
|8525|8525|03-07-2018| 95.3000030517578|  97.1999969482422|1060600| 97.86000061035156| 95.45999908447266|90.76885223388672|SWKS|
|8526|8526|05-07-2018|             96.5|              96.5|2248500|             99.75| 99.66000366210938|94.76245880126952|SWKS|
|8527|8527|06-07-2018|98.58999633789062| 99.41999816894531|1438400| 100.4499969482422|100.12999725341795| 95.2093505859375|SWKS|
|8528|8528|09-07-2018| 99.2699966430664|100.51000213623048| 994200|100.63999938964844|100.4300003

In [156]:
# reformatting the date column in crypto index to match the style of the stocks index
# 2018-07-01 23:59:59
def reformat_date(date_time_str):
    from datetime import datetime
    date = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
    string = date.strftime('%d-%m-%Y')
    return string
    
crypto10 = crypto_index_final_df.rdd.map(lambda x: (reformat_date(x[0]),x[1],x[2]))
crypto10_df = crypto10.toDF(('date', 'avg_price_change_percent', 'avg_tmv_change_percent'))
crypto10_df.show(5)

+----------+------------------------+----------------------+
|      date|avg_price_change_percent|avg_tmv_change_percent|
+----------+------------------------+----------------------+
|01-07-2018|     -0.3545608063329723|                   0.0|
|02-07-2018|       5.121125072620959|     9.193665598572519|
|03-07-2018|     -1.7263876931070241|      3.88579704445608|
|04-07-2018|      0.4605982027758544|   -17.906456882074515|
|05-07-2018|     -0.7984669906309387|     5.925269446030441|
+----------+------------------------+----------------------+
only showing top 5 rows



In [152]:

# checking if we have the same dates across all stock indices
dates = nasdaq100_df.rdd.map(lambda x: x[0]).collect()
a = set(nasdaq100_df.rdd.map(lambda x: x[0]).collect())
b = set(sp100_df.rdd.map(lambda x: x[0]).collect())
c = set(nasdaq100Tech_df.rdd.map(lambda x: x[0]).collect())
print(len(a))
print(a==c and a==b and a==b)
# crypto10 = crypto10_df.rdd.filter(lambda x: x[0] in dates)
# crypto10_df = crypto10.toDF(('date', 'avg_price_change_percent', 'avg_tmv_change_percent'))

756
True


In [154]:
crypto10_df.show(5)

+----------+------------------------+----------------------+
|      date|avg_price_change_percent|avg_tmv_change_percent|
+----------+------------------------+----------------------+
|02-07-2018|       5.121125072620959|     9.193665598572519|
|03-07-2018|     -1.7263876931070241|      3.88579704445608|
|05-07-2018|     -0.7984669906309387|     5.925269446030441|
|06-07-2018|     -0.6435487855691865|    -12.91041562423435|
|09-07-2018|      -1.453540445159325|    -5.414889467143977|
+----------+------------------------+----------------------+
only showing top 5 rows



In [155]:
# Final Data Frames for Analysis
# nasdaq100_df, sp100_df, nasdaq100Tech_df, crypto10_df

# Joining the datasets on the date column (inner join)
# using sparksql
# weekends and holidays dont have stock market data to relate the crypto currency data with
# selecting only those dates that are also in the stock market data
# leaving out the rest of the dates in the crypto would not be an issue as we are looking into the daily trading trends

# pcp columns denote the daily average price change percentage for each index
# tmvcp columns denote the daily average tmv price chance percentage for each index

s

